<a href="https://colab.research.google.com/github/veda-sunkara/StreetToCloud/blob/master/FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
LR = 1e-3
EPOCHS = 1000
EPOCHS_PER_UPDATE = 1
RUNNAME = "1e3_flood_0"

In [16]:
from google.colab import auth
auth.authenticate_user()

!curl https://sdk.cloud.google.com | bash

!gcloud init

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0  19260      0 --:--:-- --:--:-- --:--:-- 19260
######################################################################## 100.0%
Running install script from: /tmp/tmp.uKwmrnLjd5/install_google_cloud_sdk.bash
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
######################################################################## 100.0%

mkdir -p /root
"/root/google-cloud-sdk" already exists and may contain out of date files.
Remove /root/google-cloud-sdk or select a new installation directory, then run again.
Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: veda@clou

In [17]:
import torch
from torchvision import transforms
import torchvision.transforms.functional as F
import random

class InMemoryDataset(torch.utils.data.Dataset):
  
  def __init__(self, data_list, preprocess_func):
    self.data_list = data_list
    self.preprocess_func = preprocess_func
  
  def __getitem__(self, i):
    return self.preprocess_func(self.data_list[i])
  
  def __len__(self):
    return len(self.data_list)


def processAndAugment(data):
  (x,y) = data
  im,label = x.copy(), y.copy()

  # convert to PIL for easier transforms
  im1 = Image.fromarray(im[0])
  im2 = Image.fromarray(im[1])
  label = Image.fromarray(label.squeeze())

  # Get params for random transforms
  i, j, h, w = transforms.RandomCrop.get_params(im1, (256, 256))
  
  im1 = F.crop(im1, i, j, h, w)
  im2 = F.crop(im2, i, j, h, w)
  label = F.crop(label, i, j, h, w)
  if random.random() > 0.5:
    im1 = F.hflip(im1)
    im2 = F.hflip(im2)
    label = F.hflip(label)
  if random.random() > 0.5:
    im1 = F.vflip(im1)
    im2 = F.vflip(im2)
    label = F.vflip(label)
  
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  im = torch.stack([transforms.ToTensor()(im1).squeeze(), transforms.ToTensor()(im2).squeeze()])
  im = norm(im)
  label = transforms.ToTensor()(label).squeeze()
  if torch.sum(label.gt(.003) * label.lt(.004)):
    label *= 255
  label = label.round()

  return im, label


def processTestIm(data):
  (x,y) = data
  im,label = x.copy(), y.copy()
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  #label[0][0][0] = 255
  
  # convert to PIL for easier transforms
  im_c1 = Image.fromarray(im[0]).resize((512,512))
  im_c2 = Image.fromarray(im[1]).resize((512,512))
  label = Image.fromarray(label.squeeze()).resize((512,512))

  im_c1s = [F.crop(im_c1, 0, 0, 256, 256), F.crop(im_c1, 0, 256, 256, 256),
            F.crop(im_c1, 256, 0, 256, 256), F.crop(im_c1, 256, 256, 256, 256)]
  im_c2s = [F.crop(im_c2, 0, 0, 256, 256), F.crop(im_c2, 0, 256, 256, 256),
            F.crop(im_c2, 256, 0, 256, 256), F.crop(im_c2, 256, 256, 256, 256)]
  labels = [F.crop(label, 0, 0, 256, 256), F.crop(label, 0, 256, 256, 256),
            F.crop(label, 256, 0, 256, 256), F.crop(label, 256, 256, 256, 256)]


  ims = [torch.stack((transforms.ToTensor()(x).squeeze(),
                    transforms.ToTensor()(y).squeeze()))
                    for (x,y) in zip(im_c1s, im_c2s)]
  ims = [norm(im) for im in ims]
  ims = torch.stack(ims)
  labels = [(transforms.ToTensor()(label).squeeze()) for label in labels]
  labels = torch.stack(labels)
  if torch.sum(labels.gt(.003) * labels.lt(.004)):
    labels *= 255
  labels = labels.round()
  return ims, labels

In [18]:
!pip install rasterio

!gsutil cp gs://cnn_chips/permanent_water_data.csv .
!gsutil cp gs://cnn_chips/permanent_water_test_data.csv .
!gsutil cp gs://cnn_chips/permanent_water_train_data.csv .
!gsutil cp gs://cnn_chips/permanent_water_validation_data.csv .

Copying gs://cnn_chips/permanent_water_data.csv...
/ [1 files][102.2 KiB/102.2 KiB]                                                
Operation completed over 1 objects/102.2 KiB.                                    
Copying gs://cnn_chips/permanent_water_test_data.csv...
/ [1 files][ 20.5 KiB/ 20.5 KiB]                                                
Operation completed over 1 objects/20.5 KiB.                                     
Copying gs://cnn_chips/permanent_water_train_data.csv...
/ [1 files][ 61.2 KiB/ 61.2 KiB]                                                
Operation completed over 1 objects/61.2 KiB.                                     
Copying gs://cnn_chips/permanent_water_validation_data.csv...
/ [1 files][ 20.5 KiB/ 20.5 KiB]                                                
Operation completed over 1 objects/20.5 KiB.                                     


In [19]:
import csv
from PIL import Image
import rasterio
import numpy as np
import os
BASEDIR = ''

def getArr(fname):
  return rasterio.open(BASEDIR + fname).read()


def download_perm_water_data_from_file(fname):
  with open(fname) as f:
    data_fnames = [tuple(line) for line in csv.reader(f)]
  i = 0
  data = []
  for (x,y) in data_fnames:
    arr_x, arr_y = getArr(x), getArr(y)
    if np.sum((arr_x != arr_x)) == 0:
      ignore = (arr_y == -1)
      ignore = ((np.uint8(ignore) * -1) * 256) + 1
      arr_y *= ignore
      data.append((arr_x, arr_y))
      i+=1
      print(i)
    else:
      print("skipping nan")
  return data

def download_perm_train_data():
  TRAINING_DATA_FILE = BASEDIR + 'permanent_water_train_data.csv'
  return download_perm_water_data_from_file(TRAINING_DATA_FILE)

def download_perm_valid_data():
  VALID_DATA_FILE = BASEDIR + 'permanent_water_validation_data.csv'
  return download_perm_water_data_from_file(VALID_DATA_FILE)

def download_perm_test_data():
  TEST_DATA_FILE = BASEDIR + 'permanent_water_test_data.csv'
  return download_perm_water_data_from_file(TEST_DATA_FILE)

In [20]:
!cd /home

!mkdir /files3
!mkdir /files3/S1
!mkdir /files3/NoQC

mkdir: cannot create directory ‘/files3’: File exists
mkdir: cannot create directory ‘/files3/S1’: File exists
mkdir: cannot create directory ‘/files3/NoQC’: File exists


In [21]:
!gsutil -m rsync gs://cnn_chips/S1 /files3/S1/
!gsutil -m rsync gs://cnn_chips/NoQC /files3/NoQC/

Building synchronization state...
Starting synchronization...
Building synchronization state...
Starting synchronization...
Copying gs://cnn_chips/NoQC/...
Skipping attempt to download to filename ending with slash
(/files3/NoQC/). This typically happens when using gsutil to download
from a subdirectory created by the Cloud Console
(https://cloud.google.com/console)/ [0/1 files][    0.0 B/   11.0 B]   0% Done                                    


In [22]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!sudo mkdir files4
!gcsfuse --implicit-dirs cnn_chips files4

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  21766      0 --:--:-- --:--:-- --:--:-- 21766
OK
12 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.30.0).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Using mount point: /files4
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [23]:
from time import time

def getArrFlood(fname):
  return rasterio.open(fname).read()

def download_flood_water_data_from_list(l):
  i= 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (im_fname, mask_fname) in l:
    if not os.path.exists(os.path.join("files3/", im_fname)):
      print(os.path.join("files3/", im_fname))
      continue
    arr_x = np.nan_to_num(getArrFlood(os.path.join("files3/", im_fname)))
    arr_y = getArrFlood(os.path.join("files3/", mask_fname))
    ignore = (arr_y == -1)
    ignore = ((np.uint8(ignore) * -1) * 256) + 1
    arr_y *= ignore
    arr_y = np.uint8(getArrFlood(os.path.join("files3/", mask_fname)))
    if np.sum((arr_y != arr_y)) == 0:
      arr_x = np.clip(arr_x, -50, 1)
      arr_x = (arr_x + 50) / 51
      if i % 100 == 0:
        print(i)
        print(im_fname, mask_fname)
      i += 1
      flood_data.append((arr_x,arr_y))
    else:
      print("skipping nan")
  print(i)
  return flood_data


def load_flood_train_data():
  basedir = "files4/"
  fname = "files3/flood_train_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_weak_flood_train_data():
  basedir = "files4/"
  files = [(os.path.join("S1_NoQC", x[1]), os.path.join("NoQC", x[0])) for x in zip(sorted(os.listdir("files3/NoQC")), sorted(os.listdir("files3/S1_NoQC")))]
  files = [x for x in files if "Bolivia" not in x[0]]
  print(files[0:10])
  return download_flood_water_data_from_list(files)

def load_flood_valid_data():
  basedir = "files4/"
  fname = "files3/flood_valid_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  print(fname, "files!")
  return download_flood_water_data_from_list(fname)

def load_flood_test_data():
  basedir = "files4/"
  fname = "files3/flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_flood_bolivia_test_data():
  basedir = "files4/"
  fname = "files3/flood_bolivia_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

In [23]:
cd ../../

In [26]:
train_data = download_perm_train_data()
train_dataset = InMemoryDataset(train_data, processAndAugment)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=None,
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
train_iter = iter(train_loader)

RasterioIOError: ignored